In [5]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('pbp_test.csv',index_col=0)

def str_to_tuple(s):
    return tuple(s.strip('() ').replace(' ', '').split(','))

df['h_lineup'] = df.h_lineup.apply(lambda x: str_to_tuple(x))
df['a_lineup'] = df.a_lineup.apply(lambda x: str_to_tuple(x))

eventdict = {1:'FGM',
             2:'FGA',
             3:'FTA',
             4:'TRB',
             5:'TURNOVER',
             6:'FOUL',
             8:'SUB',
             9:'TIMEOUT',
             10:'JUMP_BALL',
             12:'Q_START',
             13:'Q_END',
             18:'REPLAY'             
            }
cols = ['game_id', 'lineup','player_id','PTS','FGM', 'FGA', '3PM', '3PA', 'FTM', 'FTA', 'REB', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'MIN', 'POSS']


In [15]:
#allll this ugly stuff just to get a column called 'rebound_type'
mask = (df.eventmsgtype == 4) & (~df.homedescription.isnull() | ~df.visitordescription.isnull())
pattern = r'Off:(\d+) Def:(\d+)'
#getting rebound counts from play description and filling nulls, and casting to int
df[['offensive_rebounds', 'defensive_rebounds']] = df['homedescription'].str.extract(pattern)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df['visitordescription'].str.extract(pattern)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].fillna(0)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].astype(int)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df[['offensive_rebounds_v', 'defensive_rebounds_v']].fillna(0)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df[['offensive_rebounds_v', 'defensive_rebounds_v']].astype(int)

df.sort_values(by=['player1_id','period','total_elapsed_time','eventnum'], inplace=True)

#comparing reb total to previous to check what type of rebound occurred
df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)
df['rebound_type'] = ''
df.loc[(df.defensive_rebounds > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.loc[(df.defensive_rebounds_v > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds_v > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.drop([ 'offensive_rebounds', 'defensive_rebounds',
       'offensive_rebounds_v', 'defensive_rebounds_v', 'def_reb_previous',
       'off_reb_previous'],inplace=True,axis=1)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_31776\3264647376.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_31776\3264647376.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)


In [21]:
fga_filter = (df.eventmsgtype==1) | (df.eventmsgtype==2)
two_pt_filter = (df.eventmsgtype==1) & ~((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_pt_filter = (df.eventmsgtype==1) & ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_a_filter = ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
ft_filter = (df.eventmsgtype==3) & ((df.homedescription.str.contains('PTS',na=False)) | (df.visitordescription.str.contains('PTS',na=False)))
fta_filter = df.eventmsgtype==3 
assist_filter = (df.eventmsgtype==1) & ~df.player2_name.isnull()

orb_filter = df.rebound_type == 'ORB'
drb_filter = df.rebound_type == 'DRB'
trb_filter = (df.rebound_type == 'DRB') | (df.rebound_type == 'ORB')
stl_filter = ''
blk_filter = ''
to_filter = ''
foul_filter = ''

In [22]:
df.columns

Index(['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period',
       'pctimestring', 'homedescription', 'neutraldescription',
       'visitordescription', 'score', 'scoremargin', 'player1_id',
       'player1_name', 'player1_team_abbreviation', 'player2_id',
       'player2_name', 'player2_team_abbreviation', 'player3_id',
       'player3_name', 'player3_team_abbreviation', 'timeinseconds',
       'play_elapsed_time', 'total_elapsed_time', 'max_time', 'time_remaining',
       'h_lineup', 'a_lineup', 'rebound_type'],
      dtype='object')

In [23]:
def get_stat_df(pbp_df,filter,stat_type,stat_value,player_num='1'):
    filtered_df = pbp_df.loc[filter][['game_id','eventnum',f'player{player_num}_id',f'player{player_num}_name','h_lineup']]
    filtered_df['stat'] = stat_type
    filtered_df['value'] = stat_value
    return filtered_df.values.tolist()

#results is a list of every stat credit for every play
results = get_stat_df(df,two_pt_filter,'PTS',2)
results += get_stat_df(df,three_pt_filter,'PTS',3)
results += get_stat_df(df,ft_filter,'PTS',1)

results += get_stat_df(df,fga_filter,'FGA',1)
results += get_stat_df(df,fta_filter,'FTA',1)
results += get_stat_df(df,three_a_filter,'3PA',1)

results += get_stat_df(df,trb_filter,'TRB',1)
results += get_stat_df(df,drb_filter,'DRB',1)
results += get_stat_df(df,orb_filter,'ORB',1)

results += get_stat_df(df,assist_filter,'AST',1,player_num='2')
results[-1]

[41800237,
 654,
 1627750,
 'Jamal Murray',
 ('200794', '203914', '203999', '1627750', '1628470'),
 'AST',
 1]

In [24]:
play_credit_df = pd.DataFrame(results,columns=['game_id','eventnum','player_id','player_name','lineup','stat_type','stat_value'])
play_credit_df.groupby(by=['player_id', 'stat_type','player_name'])['stat_value'].sum()

player_id  stat_type  player_name 
200794     3PA        Paul Millsap     2
           AST        Paul Millsap     1
           DRB        Paul Millsap     6
           FGA        Paul Millsap    13
           FTA        Paul Millsap     6
                                      ..
1628470    FGA        Torrey Craig     5
           FTA        Torrey Craig     5
           ORB        Torrey Craig     4
           PTS        Torrey Craig     8
           TRB        Torrey Craig     8
Name: stat_value, Length: 121, dtype: int64

In [38]:
game_play_agg = play_credit_df.groupby(by=['lineup','player_id', 'stat_type','player_name'],as_index=False)['stat_value'].sum()
game_play_agg

,lineup,player_id,stat_type,player_name,stat_value
0,"(200794, 203115, 203486, 1627736, 1628420)",200794,DRB,Paul Millsap,2
1,"(200794, 203115, 203486, 1627736, 1628420)",200794,FGA,Paul Millsap,4
2,"(200794, 203115, 203486, 1627736, 1628420)",200794,FTA,Paul Millsap,2
3,"(200794, 203115, 203486, 1627736, 1628420)",200794,ORB,Paul Millsap,1
4,"(200794, 203115, 203486, 1627736, 1628420)",200794,PTS,Paul Millsap,1
...,...,...,...,...,...
304,"(203486, 203914, 203999, 1627750, 1628470)",1627750,ORB,Jamal Murray,1
305,"(203486, 203914, 203999, 1627750, 1628470)",1627750,PTS,Jamal Murray,2
306,"(203486, 203914, 203999, 1627750, 1628470)",1627750,TRB,Jamal Murray,2
307,"(203486, 203914, 203999, 1627750, 1628470)",1628470,3PA,Torrey Craig,1


In [39]:
pivoted_game = game_play_agg.pivot(index=['lineup', 'player_id', 'player_name'], columns='stat_type', values='stat_value').fillna(0).reset_index()
pivoted_game

stat_type,lineup,player_id,player_name,3PA,AST,DRB,FGA,FTA,ORB,PTS,TRB
0,"(200794, 203115, 203486, 1627736, 1628420)",200794,Paul Millsap,0.0,0.0,2.0,4.0,2.0,1.0,1.0,3.0
1,"(200794, 203115, 203486, 1627736, 1628420)",202323,Evan Turner,0.0,0.0,3.0,2.0,0.0,0.0,2.0,3.0
2,"(200794, 203115, 203486, 1627736, 1628420)",203081,Damian Lillard,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,"(200794, 203115, 203486, 1627736, 1628420)",203115,Will Barton,1.0,0.0,1.0,6.0,0.0,1.0,4.0,2.0
4,"(200794, 203115, 203486, 1627736, 1628420)",203468,CJ McCollum,0.0,1.0,3.0,3.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
83,"(203486, 203914, 203999, 1627750, 1628470)",203486,Mason Plumlee,0.0,0.0,2.0,0.0,2.0,0.0,1.0,2.0
84,"(203486, 203914, 203999, 1627750, 1628470)",203918,Rodney Hood,0.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0
85,"(203486, 203914, 203999, 1627750, 1628470)",203999,Nikola Jokic,0.0,0.0,1.0,4.0,0.0,0.0,2.0,1.0
86,"(203486, 203914, 203999, 1627750, 1628470)",1627750,Jamal Murray,1.0,1.0,1.0,5.0,0.0,1.0,2.0,2.0


In [40]:
 game_play_agg.pivot(index=['player_id', 'player_name'], columns='stat_type', values='stat_value').fillna(0).reset_index()

ValueError: Index contains duplicate entries, cannot reshape